# Esercitazione 2

Sempre partendo dai dati sulle definizioni, si richiede di provare a costruire un sistema che utilizzi la molteplicità delle definizioni per risalire al termine "target" in maniera automatica. Non si richiede di "indovinare" ogni termine, ma di avvicinarsi (almeno semanticamente) alla risposta. Provare più soluzioni, includendo meccanismi di filtro delle definizioni (ad es. escludendo quelle meno informative o con caratteristiche particolari), di ricerca nell'albero tassonomico di WordNet (provando a partire da candidati "genus", secondo il principio Genus-Differentia), ecc.

In [185]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

## Pre-processing

In [237]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clear_sentence(sentence, only_nouns = False):
    tokens = nltk.word_tokenize(sentence)
    
    if only_nouns:
        #mantengo solo i sostantivi
        tokens = nltk.pos_tag(tokens)
        tokens = [word for word in tokens if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']]
        tokens = [word[0] for word in tokens]

    tokens = [token for token in tokens if token not in string.punctuation] #tolgo la punteggiatura
    tokens = [token.lower() for token in tokens] # sostituisco le maiuscole con le minuscole
    tokens = [token for token in tokens if token not in stop_words] # rimuovo le stop words
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # lemmatizzo
    
    return tokens

## Metodi di supporto

In [226]:
def get_context(definitions):
    context = set()
    for definition in definitions:
        context = context.union(set(definition))
    return context

def simplified_lesk(word, context):
    best_sense = None
    max_overlap = 0

    for sense in wn.synsets(word, pos='n'):
        signature = set(clear_sentence(sense.definition())).union(set(clear_sentence(' '.join(sense.examples()))))
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    
    return best_sense


## Ottenimento del geneus

In [217]:
'''
Restituisce come geneus il token che compare più volte nelle definizioni
'''
def get_geneus(definitions, syn = None):
    words = []
    for definition in definitions:
        words += definition

    # se ho il synset aggiungo alle parole anche la definizione del synset
    if syn is not None:
        words += clear_sentence(syn.definition())

    return nltk.FreqDist(words).most_common(1)[0][0]

## Approccio 1: recursive simplified lesk

Partendo dal geneus, si prendono tutti i suoi synset, e ad ognuno di essi si applica il recursive_simplified_lesk. La caratteristica
di questo algoritmo è che non si ferma al primo livello di iponimi del geneus, ma per ognuno di essi scende di un certo numero di livelli (iperparametro da specificare). Così facendo vengono analizzati un numero maggiore di synsets così da avere più probabilità di ottenere il synset corretto.

In [219]:
'''
Variante ricorsiva del lesk, che permette di scendere di livello nella
gerarchia del geneus
'''
def recursive_simplified_lesk(level, best_sense, current_syn, max_overlap, context):
    if level == 0:
        return [best_sense, max_overlap]
    else:
        for syn in current_syn.hyponyms():
            signature = set(clear_sentence(syn.definition())).union(clear_sentence(' ' .join(syn.examples())))
            overlap = len(context.intersection(signature))
            if overlap >= max_overlap:
                max_overlap = overlap
                best_sense = syn

        return recursive_simplified_lesk(level - 1, best_sense, current_syn, max_overlap, context)

'''
Metodo per predire il token dato un insieme di definizioni
'''
def predict_token(definitions, geneus):
    best_sense = None
    max_overlap = 0

    # ottengo il contesto
    context = get_context(definitions)

    # per ogni synset del geneus cerco il synset con il massimo overlap 
    # con il contesto, andando ad usare una variante personalizzata
    # del simplified lesk
    for syn_geneus in wn.synsets(geneus, pos='n'):    
        sense, overlap = recursive_simplified_lesk(5, best_sense, syn_geneus, max_overlap, context)
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense

    return best_sense

## Approccio 2: rimozione definizioni poco informative + recursive simplified lesk

In [241]:
def remove_uninformative_definitions(definitions, syn):
    syn_definition = clear_sentence(syn.definition())


    defs_overlap = []
    # per ogni definizione calcolo l'overlap con la definizione del synset
    for definition in definitions:
        defs_overlap.append(len(set(clear_sentence(definition)).intersection(set(syn_definition))))

    # rimuovo le definizioni che hanno overlap minore
    min_overlap = min(defs_overlap)
    
    new_definitions = []
    for i in range(len(definitions)):
        if defs_overlap[i] != min_overlap:
            print(definitions[i])
            print(clear_sentence(definitions[i], only_nouns=True))
            new_definitions.append(clear_sentence(definitions[i], only_nouns=True))

    return new_definitions

## Main

### Applicazione approccio 1

In [239]:
corpus = pd.read_csv('definizioni.tsv', sep='\t', engine='python')

corpus['door'] = corpus['door'].apply(clear_sentence, only_nouns=True)
corpus['ladybug'] = corpus['ladybug'].apply(clear_sentence, only_nouns=True)
corpus['pain'] = corpus['pain'].apply(clear_sentence, only_nouns=True)
corpus['blurriness'] = corpus['blurriness'].apply(clear_sentence, only_nouns=True)

for token in corpus.columns:
    geneus = get_geneus(corpus[token])
    predicted_token = predict_token(corpus[token], geneus)

    print('token: ' + token)
    print("geneus: " + geneus)
    print("predicted token: " + predicted_token.name())

    print("\n")

token: door
geneus: room
predicted token: toilet.n.01


token: ladybug
geneus: insect
predicted token: lepidopterous_insect.n.01


token: pain
geneus: sensation
predicted token: sound.n.02


token: blurriness
geneus: image
predicted token: likeness.n.02




In [258]:
wn.synset("pain.n.02").definition()

'emotional distress; a fundamental feeling that people try to avoid'

### Applicazione approccio 2

In [255]:
corpus = pd.read_csv('definizioni.tsv', sep='\t', engine='python')


for token in ['door', 'ladybug', 'pain', 'blurriness']:
    definitions = []
    for definition in corpus[token]:
        definitions.append(clear_sentence(definition))

    syn = simplified_lesk(token, get_context(definitions))
    new_definitions = remove_uninformative_definitions(corpus[token], syn)
    geneus = get_geneus(new_definitions, syn)
    predicted_token = predict_token(new_definitions, geneus)

    print('Token: ', token)
    print('Geneus: ', geneus)
    print("predicted token: " + predicted_token.name())
    print('\n')


Token:  door
Geneus:  room
predicted token: toilet.n.01


Token:  ladybug
Geneus:  insect
predicted token: lepidopterous_insect.n.01


Token:  pain
Geneus:  feeling
predicted token: pain.n.02


Token:  blurriness
Geneus:  image
predicted token: visual_image.n.01


